In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import gensim
import re, string
from scipy.sparse import csr_matrix, hstack
import sklearn
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words

%matplotlib inline

In [ ]:
hours = pd.read_excel('ActivitiesGangHours.xlsx', index = 'Activity ID')
hours.drop('Activities Description', axis=1, inplace=True) #TODO: revise!!!

In [ ]:
df = pd.read_excel('ActivityConstrains.xlsx', index = 'Activity ID')
df.drop(['Name','Unnamed: 7'], axis=1,inplace=True)
df = df.dropna()###disabel if instructions how to handle provided
df["Activity ID"]=df["Activity ID"].astype(int)

In [ ]:
df.columns = ['ID','Description','Element','Extractor','Operation','Value','Unit']

In [ ]:
df.loc[df.Unit=='mm','Value']/=1000
df.loc[df.Unit=='mm','Unit']='m'

In [ ]:
vectorizer = CountVectorizer(min_df=0, token_pattern=r"\b\w+\b")
vectorizer.fit(df.Unit.unique())
uns = vectorizer.transform(df.Unit)

index = [i for i in range(1, uns.shape[0]+1)]
units = pd.SparseDataFrame(uns, index=index, columns=vectorizer.get_feature_names())
units.fillna(0, inplace=True)

In [ ]:
vectorizer = CountVectorizer()
els = vectorizer.fit_transform(df['Element (tag)'])

elements = pd.SparseDataFrame(els, index=index, columns=vectorizer.get_feature_names())
elements.fillna(0, inplace= True)

In [ ]:
df['semicolon']= df['Activities Description'].apply(lambda x: x.count(';'))
df['commas']= df['Activities Description'].apply(lambda x: x.count(','))

In [ ]:
number_list = ["zero","one","two","three","four","five","six","seven"," eight","nine"]
plural_list = ['hasdkjha', 'single','double','tripple']
def extract_numbers(s):  
    for i in range(len(number_list)):
        if (number_list[i] in s) or ((i<len(plural_list)) and  (plural_list[i] in s)):
            return i        
    return 1       



In [ ]:
#df['txt_nmbr'] = df['Activities Description'].apply(extract_numbers)
df['Description'].apply(extract_numbers).unique()

In [ ]:
df.apply(lambda x: x['Description'].replace(x.Unit,'').replace(str(x.Value),''), axis =1)

In [ ]:
df.head()

In [ ]:
df['Activities Description'].apply(lambda x: regex.sub(" ",x))

In [ ]:
tfidf = TfidfVectorizer()
descr =tfidf.fit_transform(df['Activities Description'])

In [ ]:
print(sp.shape, elements.shape, units.shape, descr.shape)

In [ ]:
j = df.merge(hours, how='left', left_on='Activity ID', right_on='ID').drop(['Activities Description_y','ID'],axis=1)

In [ ]:
j[(j['Element (tag)']=='wall') & (j['Extractor']=='radius')].plot.scatter('Value','Gang Hours')

In [ ]:
colors= {'m':'Blue','m':'Red'}

In [ ]:
desc = df['Description']

In [ ]:
#desc.apply(lambda x: x.rsplit('\n')[0]).apply(lambda x: x.rsplit(';')[0]).unique()

In [ ]:
regex = re.compile('[' +re.escape(string.punctuation) + '\d\.\\r\\t\\n]')
descr = desc.apply(lambda x: regex.sub(" ",x))

In [ ]:
stopwords = df.Extractor.unique().tolist()
stopwords += df['Element'].unique().tolist()
stopwords += df.Unit.unique().tolist()
stopwords += stop_words.ENGLISH_STOP_WORDS
stopwords += ['exceeding','over','thickness', 'size','like','mm']
len(stopwords)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word',stop_words=frozenset(stopwords))
vectorizer.fit(descr)

In [ ]:
descr_df = pd.DataFrame(vectorizer.transform(descr).todense(),columns=list(vectorizer.vocabulary_.keys())).fillna(0.0)

In [ ]:
descr_df.index = df.index
descr_df['ID']=df.ID.astype(int)
descr_df = descr_df.groupby(by='ID').mean().reset_index()

In [ ]:
ds = descr_df.merge(sp,on='ID',how='left')

In [ ]:
ds.describe()


In [ ]:
op_cat_df = df[df.Operation!='=']
op_cat_df['cat_value'] = op_cat_df[['Element','Extractor','Value']].groupby(by=['Element','Extractor'])['Value'].transform(lambda x: pd.factorize(x, sort=True)[0])
op_cat_df['cat_value']+=1 ##To leave zeros for non-interval values
op_cat_df.loc[op_cat_df['Operation'].isin(['>','>=','=>']),'cat_value']+=1

In [ ]:
#df['cat_value']=0
df.loc[op_cat_df.index,'cat_value']=op_cat_df['cat_value']
df.loc[op_cat_df.index,'Value']=0

In [ ]:
df['Op']=0
df.loc[df.Operation=='=','Op']=1

In [ ]:
df['ElExtOp']=df.Element +  df.Extractor + df.Op.astype(str)
df.drop(['Element','Extractor','Op','Operation','Unit'],axis=1,inplace=True)

In [ ]:
df.Value +=df.cat_value

In [ ]:
sp = df.pivot(columns='ElExtOp', values='Value').fillna(0.0)
sp['ID']=df.ID

In [ ]:
sp = sp.groupby(by='ID').sum().reset_index()

In [ ]:
sp.shape